## qpth Julia implementation

- A solver for QP of form
 - min z'Qz/2 + q'z
 - s.t. A z = b, G z <= h

In [9]:
using LinearAlgebra;
using Test;

In [2]:
function get_sizes(G,A=Nothing)
    nineq, nz = size(G)
    
    if A != Nothing
        if length(A) > 0 
            neq = size(A)[1]
        else 
            neq = 0
        end
    else
        neq = Nothing
    end
    
    return nineq, nz, neq
end

get_sizes (generic function with 2 methods)

In [6]:
@test get_sizes(ones(4,3), ones(2,2)) == (4,3,2)
@test get_sizes(ones(4,3))            == (4, 3, Nothing)
@test get_sizes(ones(3,2),[])         == (3, 2, 0)

Test Passed

In [81]:
function lu_hack(X)
    # wasnt getting same pivoted LU decomposition as torch
    # using native implementation for now
    X_copy = copy(X)
    data, pivots = LAPACK.getrf!(X_copy) # mutates the matrix
    return data, pivots
end

lu_hack (generic function with 1 method)

In [87]:
@test lu_hack([[1.,2.] [2.,2.]]) == ([[2., .5] [2., 1.]], [2,2])

Test Passed

In [91]:
function pre_factor_kkt(Q,G,A)
    nineq, nz, neq = get_sizes(G, A)
    
    # S = [ A Q^{-1} A^T    A Q^{-1} G^T         ]
    #     [ G Q^{-1} A^T    G Q^{-1} G^T + D^{-1}]
    # First find inv(D)
    # Then compute a partial LU decomposition of S
    
    # pivoted LU decomposition; will work for non singular matrices too
    Q_LU = lu_hack(Q)  
    
#     U_S = zeros(neq + nineq, neq + nineq) # TODO: append .type_as(Q)
    
#     G_invQ_GT = G*(inv(U_Q.L*U_Q.L')*G')
#     R = G_invQ_GT
    
#     if neq > 0
#         invQ_AT = inv(U_Q.L*U_Q.L')*A'
#         A_invQ_AT = A*invQ_AT
#         G_invQ_AT = G*invQ_AT

#         U11 = cholesky(A_invQ_AT)
#         U12 =  U11.L' \ G_invQ_AT'
#         U_S[1:neq,1:neq] = ones(neq,neq)*U11.L[1]
#         U_S[1:neq,neq:neq+nineq] = ones(neq,nineq+1)*U12[1]
#         R = R - U12'*U12
#     end
    
#     return U_Q.L, U_S, R
    return Q_LU
end 

pre_factor_kkt (generic function with 1 method)

In [126]:
Q = [[2.0,-1.0,0.0] [-1.0,2.0,-1.0] [0.0,-1.0,2.0]]'
G = [[1.0, 2.0, 1.0] [1.0, 2.0, 1.0]]'
A = [[2.0 0.0 -1.0]]'
# pre_factor_kkt(Q,G,A)
G * (G' \ Q)

DimensionMismatch: DimensionMismatch("A has dimensions (2,3) but B has dimensions (2,3)")

## Solve a QP using JuMP

In [8]:
using JuMP, Ipopt;

In [9]:
model = Model(Ipopt.Optimizer);

# suppressing verbose logs for Ipopt
MOI.set(model, MOI.RawParameter("print_level"), 1);

In [10]:
@variable(model, x[1:2]);

In [11]:
Q = [6 4; 0 2];
q = [1; 6];
G = [-2 -3; -1 0; 0 -1];
h = [-4; 0; 0];
# ignoring equality constraint variables A, b for now

In [12]:
obj_expr = x'*Q*x/2 + q'*x;

In [13]:
@objective(model, Min, obj_expr)

3 x[1]² + 2 x[2]*x[1] + x[2]² + x[1] + 6 x[2]

In [14]:
@constraint(model, G*x .<= h)

3-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 -2 x[1] - 3 x[2] ≤ -4.0
 -x[1] ≤ 0.0            
 -x[2] ≤ 0.0            

In [15]:
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [16]:
value.(x)

2-element Array{Float64,1}:
 0.5000000028537452
 0.99999998504258  

## General Construct to solve a QP
Using JuMP with Ipopt solver


 min | `z'Qz/2 + q'z` 
--------|------------
s.t. | `A z = b`
     | `G z <= h`

In [68]:
nz = 10  # no of variables
nineq = 10  # no of inequalities
neq = 0; # no of equalities

In [69]:
# generate the matrices
L = rand(nz, nz)
Q = L*L' .+ 1e-3*Matrix{Float64}(I, nz, nz) 
G = randn(nineq, nz)
z0 = randn(nz)
s0 = rand(nineq)
p = randn(nz)
h = G*z0 + s0
A = randn(neq, nz)
b = A*z0;

In [112]:
using JuMP, Ipopt;

function forward_single_np(Q, p, G, h, A, b)
    nz, neq, nineq = size(p)[1], size(A)[1], size(G)[1]

    model = Model(Ipopt.Optimizer)
    MOI.set(model, MOI.RawParameter("print_level"), 1) # suppress logs for Ipopt

    @variable(model, z_[1:nz])
    
    obj_expr = z_'*Q*z_/2 + p'*z_
    
    @objective(model, Min, obj_expr)
    
    
    # donno whether Jump supports adding slack variables
    # adding them manually
    if nineq > 0
        @variable(model, slacks[1:nineq])
        @constraint(model, ineqCon, G*z_ + slacks .== h)
        @constraint(model, slackCon, slacks .>= 0)
    end
    
    if neq > 0
        @constraint(model, eqCon, A*z_ .== b)
    end
    
    optimize!(model)
    @assert termination_status(model) in [MOI.OPTIMAL, MOI.LOCALLY_SOLVED]
    
    zhat = value.(z_)
    
    if nineq > 0
        lam = dual.(ineqCon)
        slacks = value.(slacks)
    else
        lam, slacks = Nothing, Nothing
    end
    
    if neq > 0
        nu = dual.(eqCon)
    else
        nu = Nothing
    end
    
    return objective_value(model), zhat, nu, lam, slacks
end

forward_single_np (generic function with 1 method)

In [113]:
forward_single_np(Q, p, G, h, A, b)

(-7.2757870156567686, [-3.87251, 0.775252, 0.539828, 1.13569, -2.10715, 2.11295, -1.03799, 3.77436, -1.04709, 0.790716], Nothing, [-0.337616, -4.17226e-10, -0.234188, -1.4937e-10, -8.06203e-11, -0.0983343, -2.59631e-9, -0.27987, -0.28547, -1.21266e-10], [-7.30732e-9, 2.17884, -6.11811e-9, 6.08581, 11.2749, -7.55055e-10, 0.350146, -6.75174e-9, -6.81546e-9, 7.4961])